In [1]:
!pip install transformers

In [3]:
import torch
from torchvision import datasets
from transformers import CLIPModel, CLIPProcessor, BlipProcessor, BlipForConditionalGeneration
import pandas as pd
from PIL import Image
torch.cuda.is_available()

/home/ell/work/flickr/flickr/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [4]:

def filter_images():
    with open("Flickr8k_text/Flickr8k.token.txt", "r") as file:
        l = set()
        for line in file:
            # if any([x in line for x in ['soldier', 'weapon', 'military', 'railroad', 'barricade', 'airplane', 'fighter', 'flood', 'ruin', 'army', 'truck']]):
            l.add(line.split('\t')[0][:-2])
        print(f"Count {len(l)}")
    return l

def label_set():
    images = filter_images()
    df = pd.DataFrame(index=list(images), columns=["Caption 1", "Caption 2", "Caption 3", "Caption 4"])
    with open("Flickr8k_text/Flickr8k.token.txt", "r") as file:
        for line in file:
            if not any([x in line for x in images]):
                continue
            identifier, cap = line.strip('\n').split('\t')
            name, capNum = identifier.split('#')
            df.at[name, f"Caption {int(capNum)+1}"] = cap
    return df
df = label_set()

Count 8092


In [22]:
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")


def iter_apply(row):
    image = Image.open("Flickr8k_Dataset/Flicker8k_Dataset/" + row.name)
    with torch.no_grad():
        ins = blip_processor(image, return_tensors="pt")
        outs = blip_model.generate(**ins)
        result = blip_processor.decode(outs[0], skip_special_token=True)
        row["blip"] = result.replace("[SEP]", ".")
        ins = clip_processor(text=list(row), images=[image], return_tensors="pt", padding=True)
        outs = clip_model(**ins)
    score = outs.logits_per_image.numpy()[0]
    ind = list(row.index)
    row = row.reindex(ind + [x + " score" for x in ind])
    row.loc[[x + " score" for x in ind]] = score
    print(row)
    return row

df = df.apply(iter_apply, axis=1)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


TypeError: replace expected at least 2 arguments, got 1